# 다항 회귀 및 정규화 (Polynomial Regression & Regularization)

**학습 목표:**
- 단순 선형 회귀의 한계를 이해하고, 비선형 관계를 모델링하기 위해 **다항 회귀**를 사용합니다.
- 모델의 복잡도가 증가할 때 발생하는 **과적합(Overfitting)** 문제를 시각적으로 확인합니다.
- 과적합을 제어하기 위한 정규화(Regularization) 기법인 **Ridge(L2)** 및 **Lasso(L1)** 회귀를 적용하고 그 효과를 비교합니다.
- `scikit-learn`의 `Pipeline`을 사용하여 데이터 전처리 및 모델 학습 과정을 효율적으로 구성하는 방법을 배웁니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score

### (1) 데이터 준비 및 탐색 (EDA)
보스턴 주택 가격 데이터셋을 사용합니다. 이 중 'LSTAT'(하위 계층 비율) 특성과 'MEDV'(주택 가격) 간의 관계를 중심으로 분석하겠습니다.

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data"
col_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
housing_df = pd.read_csv(url, delim_whitespace=True, names=col_names)

# LSTAT과 MEDV만 사용
X = housing_df[['LSTAT']]
y = housing_df['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# LSTAT과 MEDV의 관계 시각화
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_train['LSTAT'], y=y_train)
plt.title('LSTAT vs MEDV (Training Data)')
plt.xlabel('LSTAT (Percentage of lower status of the population)')
plt.ylabel('MEDV (Median value of owner-occupied homes in $1000s)')
plt.show()
print("데이터가 선형 관계보다는 곡선 형태의 비선형 관계를 보이는 것을 확인할 수 있습니다.")

### (2) 다항 회귀를 이용한 비선형 관계 모델링
단순 선형 회귀와 2차 다항 회귀 모델을 비교하여 비선형 모델의 성능 향상을 확인합니다.

In [ ]:
# 1차 선형 회귀 (단순 회귀)
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_pred_linear = linear_model.predict(X_test)
rmse_linear = np.sqrt(mean_squared_error(y_test, y_pred_linear))

# 2차 다항 회귀
# make_pipeline으로 PolynomialFeatures와 LinearRegression을 연결
poly_model = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
poly_model.fit(X_train, y_train)
y_pred_poly = poly_model.predict(X_test)
rmse_poly = np.sqrt(mean_squared_error(y_test, y_pred_poly))

print(f"Linear Regression RMSE: {rmse_linear:.4f}")
print(f"Polynomial Regression (Degree 2) RMSE: {rmse_poly:.4f}")

# 결과 시각화
plt.figure(figsize=(10, 7))
plt.scatter(X['LSTAT'], y, label='Actual Data', alpha=0.5)
X_plot = np.sort(X['LSTAT'].unique()).reshape(-1, 1)
plt.plot(X_plot, linear_model.predict(X_plot), color='red', linewidth=2, label=f'Linear Fit (RMSE: {rmse_linear:.2f})')
plt.plot(X_plot, poly_model.predict(X_plot), color='green', linewidth=2, label=f'Polynomial Fit (RMSE: {rmse_poly:.2f})')
plt.title('Linear vs Polynomial Regression')
plt.xlabel('LSTAT')
plt.ylabel('MEDV')
plt.legend()
plt.show()

### (3) 과적합 문제 확인
차수(degree)를 높여 모델을 더 복잡하게 만들면 훈련 데이터는 더 잘 예측하지만, 새로운 데이터(테스트 데이터)에 대한 성능은 오히려 나빠지는 과적합 현상이 발생합니다.

In [ ]:
# 10차 다항 회귀 모델
overfit_model = make_pipeline(PolynomialFeatures(degree=10), LinearRegression())
overfit_model.fit(X_train, y_train)
y_pred_overfit = overfit_model.predict(X_test)
rmse_overfit = np.sqrt(mean_squared_error(y_test, y_pred_overfit))

# 시각화
plt.figure(figsize=(10, 7))
plt.scatter(X['LSTAT'], y, label='Actual Data', alpha=0.5)
plt.plot(X_plot, poly_model.predict(X_plot), color='green', linewidth=2, label=f'Degree 2 (RMSE: {rmse_poly:.2f})')
plt.plot(X_plot, overfit_model.predict(X_plot), color='purple', linewidth=2, label=f'Degree 10 (RMSE: {rmse_overfit:.2f})')
plt.title('Overfitting with High-Degree Polynomial')
plt.xlabel('LSTAT')
plt.ylabel('MEDV')
plt.ylim(0, 55)
plt.legend()
plt.show()
print("10차 모델은 훈련 데이터의 노이즈까지 학습하여 예측선이 매우 불안정하며, 2차 모델보다 테스트 RMSE가 높습니다.")

### (4) 정규화를 통한 과적합 제어
복잡한 모델(10차 다항)에 Ridge(L2)와 Lasso(L1) 정규화를 적용하여 과적합을 완화합니다. 정규화 모델은 특성 스케일링에 영향을 받으므로 `StandardScaler`를 파이프라인에 추가합니다.

In [ ]:
# Ridge 회귀 (alpha는 규제 강도)
ridge_model = make_pipeline(PolynomialFeatures(degree=10), StandardScaler(), Ridge(alpha=1.0))
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)
rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred_ridge))

# Lasso 회귀
lasso_model = make_pipeline(PolynomialFeatures(degree=10), StandardScaler(), Lasso(alpha=0.1))
lasso_model.fit(X_train, y_train)
y_pred_lasso = lasso_model.predict(X_test)
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))

print(f"High-degree (10) Linear RMSE: {rmse_overfit:.4f}")
print(f"High-degree (10) Ridge RMSE:   {rmse_ridge:.4f}")
print(f"High-degree (10) Lasso RMSE:   {rmse_lasso:.4f}")

# 시각화
plt.figure(figsize=(10, 7))
plt.scatter(X['LSTAT'], y, label='Actual Data', alpha=0.5)
plt.plot(X_plot, overfit_model.predict(X_plot), color='purple', linestyle='--', linewidth=2, label=f'Linear (Overfit) RMSE: {rmse_overfit:.2f}')
plt.plot(X_plot, ridge_model.predict(X_plot), color='orange', linewidth=2, label=f'Ridge (Regularized) RMSE: {rmse_ridge:.2f}')
plt.plot(X_plot, lasso_model.predict(X_plot), color='cyan', linewidth=2, label=f'Lasso (Regularized) RMSE: {rmse_lasso:.2f}')
plt.title('Effect of Regularization on Overfitting')
plt.xlabel('LSTAT')
plt.ylabel('MEDV')
plt.ylim(0, 55)
plt.legend()
plt.show()
print("Ridge와 Lasso 모델이 과적합된 Linear 모델보다 훨씬 안정적인 예측선을 그리며, 테스트 RMSE도 낮은 것을 볼 수 있습니다.")